Copy files from subfolders to directory

In [2]:
import datetime as dt
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path
from glob import iglob


In [3]:
import os
import shutil

src = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\August_2022'
dest = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Data'

for path, subdirs, files in os.walk(src):
    for name in files:
        filename = os.path.join(path, name)
        shutil.copy2(filename, dest)

convert lowercase to upper

In [151]:
import os

path = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Pilot\Data_ibutton_05092022\temp\\'

for file in os.listdir(path):
    os.rename(path + file, path + file.upper())

    

In [99]:
import os
import shutil

src = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Data'
dest = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\RH'

for path, subdirs, files in os.walk(src):
    for name in files:
        if name.startswith('H'):
            filename = os.path.join(path, name)
            shutil.copy2(filename, dest)

In [100]:
src = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Data'
dest = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\temp'

for path, subdirs, files in os.walk(src):
    for name in files:
        if name.startswith('T'):
            filename = os.path.join(path, name)
            shutil.copy2(filename, dest)

In [31]:
import datetime as dt
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path
from glob import iglob



path = r'C:\Users\ferha\OneDrive\Desktop\HIGH_Nouna\Pilot_Last\Pilot_RH' # use your path
all_files = glob.glob(path + "/*.csv")

cnt = 0
finalexcelsheet = pd.DataFrame()
for filename in all_files:
    df = pd.read_csv(filename, sep=',', index_col=None, header=0, engine='python', dayfirst=True)
    df = df.replace(to_replace='##N/A##', value=np.nan)

from pathlib import Path
p = Path(r'C:\Users\ferha\OneDrive\Desktop\HIGH_Nouna\Questionaires\New folder\HxMed\4')
df.to_csv(Path(p, 'P5_HxMed_4.csv'), index=False)


create clean version of all csv files

In [171]:
import datetime as dt
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path
from glob import iglob



path = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Pilot\Data_ibutton_05092022\temp' # use your path
all_files = glob.glob(path + "/*.csv")

cnt = 0
finalexcelsheet = pd.DataFrame()
for filename in all_files:
    df = pd.read_csv(filename, sep='§', index_col=None, header=0, engine='python', dayfirst=True)
    s1 = filename
    s2 = "T_"
    s= s1[s1.index(s2) + len(s2):]
    split_string = s.split("_")
    household = split_string[0]

    df = df.iloc[18:]
    df = df.rename(columns={ df.columns[0]: household })
    df = df[household].str.split(",")
    df = df.rename(filename)
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    df2 = pd.DataFrame(df)
    split_df1 = pd.DataFrame(df2[filename].tolist())

    
    #split_df1 = split_df1[split_df1[0] != '31/09/21 23:05:01']
    #split_df1 = split_df1[split_df1[0] != '31/09/21 23:20:01']
    #split_df1 = split_df1[split_df1[0] != '31/09/21 23:35:01']
    #split_df1 = split_df1[split_df1[0] != '31/09/21 23:50:01']
    
    
    #
    #split_df1['Dates'] = pd.to_datetime(split_df1[0], dayfirst=True).dt.date
    #split_df1['Time'] = pd.to_datetime(split_df1[0]).dt.time
    
    split_df1[household] = split_df1[[2, 3]].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)
    split_df1 = split_df1.drop([2, 3], axis=1, errors='ignore')
    split_df1[household] = split_df1[household].apply(lambda x: x.replace(',','.'))
    split_df1[household] = split_df1[household].astype('float64')
    split_df1[1] = split_df1[1].astype('string')
    for index, row in split_df1.iterrows():
        if split_df1[1][index] == 'F':
            split_df1[household][index] = (split_df1[household][index]-32)*(5/9)
            split_df1.loc[index, 1] = 'C'
    
    
    split_df1.rename(columns = {0: 'DateTime'}, inplace = True)
    
    split_df1 = split_df1.drop([1], axis=1, errors='ignore')
    
    
    split_df1['DateTime'] = pd.to_datetime(split_df1['DateTime']).dt.strftime("%d/%m/%Y %H:%M:%S")
    
    split_df1['DateTime'] = split_df1['DateTime'].astype('datetime64')
    split_df1['DateTime'] = split_df1['DateTime'].agg(lambda x : x.round('15min'))
    
    #split_df1 = split_df1.set_index('DateTime')
    split_df1 = split_df1.dropna()
    
    split_df1 = split_df1.sort_values(['DateTime'])
    


    finalexcelsheet = split_df1
    finalexcelsheet = finalexcelsheet.drop_duplicates()
    cnt +=1
    from pathlib import Path
    p = Path(r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Pilot\Data_ibutton_05092022\TEMP_CLEANSED')
    finalexcelsheet.to_csv(Path(p, 'H_' + str(household)+ '_' + str(cnt) + '.csv'), index=False)
    
    
    
    

    print(split_df1.memory_usage(index=False, deep=True))
    print(split_df1.info())
    

split_df1

DateTime    13856
0007AA      13856
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1732 entries, 0 to 1731
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DateTime  1732 non-null   datetime64[ns]
 1   0007AA    1732 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.6 KB
None
DateTime    13848
0044AA      13848
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1731 entries, 0 to 1730
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DateTime  1731 non-null   datetime64[ns]
 1   0044AA    1731 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.6 KB
None
DateTime    13848
0049AA      13848
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1731 entries, 0 to 1730
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 

,DateTime,0088AA
0,2022-08-18 13:30:00,34.629
1,2022-08-18 13:45:00,36.127
2,2022-08-18 14:00:00,36.127
3,2022-08-18 14:15:00,36.626
4,2022-08-18 14:30:00,36.626
...,...,...
1724,2022-09-05 12:30:00,38.622
1725,2022-09-05 12:45:00,39.619
1726,2022-09-05 13:00:00,39.619
1727,2022-09-05 13:15:00,39.619


getting householdIDs from csv files in given folder

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path
from glob import iglob



path = r'C:\Users\ferha\OneDrive\Desktop\HIGH_Nouna\RCT RH Update Cleansed' # use your path
all_files = glob.glob(path + "/*.csv")

cnt = 0
finalexcelsheet = pd.DataFrame()
for filename in all_files:
    df = pd.read_csv(filename, sep='§', index_col=None, header=0, engine='python', dayfirst=True)
    s1 = filename
    s2 = "T_"
    s= s1[s1.index(s2) + len(s2):]
    split_string = s.split("_")
    household = split_string[0]

    df = df.iloc[18:]
    df = df.rename(columns={ df.columns[0]: household })
    df = df[household].str.split(",")
    df = df.rename(filename)
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    df2 = pd.DataFrame(df)
    split_df1 = pd.DataFrame(df2[filename].tolist())

    #
    #split_df1['Dates'] = pd.to_datetime(split_df1[0], dayfirst=True).dt.date
    #split_df1['Time'] = pd.to_datetime(split_df1[0]).dt.time

    split_df1[household] = split_df1[[2, 3]].apply(lambda x: ','.join(x[x.notnull()]), axis = 1)
    split_df1 = split_df1.drop([ 2, 3], axis=1, errors='ignore')
    split_df1.rename(columns = {0: 'DateTime', 1: 'Unit'}, inplace = True)
    split_df1[household] = split_df1[household].apply(lambda x: x.replace(',','.'))
    split_df1[household] = split_df1[household].astype('float64')
    split_df1['Unit'] = split_df1['Unit'].astype('string')
    
    
    split_df1['DateTime'] = pd.to_datetime(split_df1['DateTime']).dt.strftime("%d/%m/%Y %H:%M:%S")
    
    split_df1['DateTime'] = split_df1['DateTime'].astype('datetime64')
    split_df1['DateTime'] = split_df1['DateTime'].agg(lambda x : x.round('15min'))
    
    split_df1 = split_df1.set_index('DateTime')
    split_df1 = split_df1.dropna()
    
    split_df1 = split_df1.sort_values(['DateTime'])
    
    finalexcelsheet = split_df1
    finalexcelsheet = finalexcelsheet.drop_duplicates()
    cnt +=1
    from pathlib import Path
    p = Path(r'C:\Users\ferha\OneDrive\Desktop\HIGH_Nouna\RCT Temp Cleansed')
    #finalexcelsheet.to_csv(Path(p, 'T_' + str(household)+ '_' + str(cnt) + '.csv'), index=False)
    
    #df =split_df1.join(df, how='outer')
    
    
print(split_df1.memory_usage(index=False, deep=True))
print(split_df1.info())
    

split_df1

In [101]:
path = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\RH'
all_files = glob.glob(path + "/*.csv")

list_hh = []

cnt = 0
finalexcelsheet = pd.DataFrame()
for filename in all_files:
    df = pd.read_csv(filename, sep='comma', index_col=None, header=0, engine='python')
    s1 = filename
    s2 = "H_"
    s= s1[s1.index(s2) + len(s2):]
    split_string = s.split("_")
    household = split_string[0]
    
    list_hh.append(str(household))
    

Count the appearance and store them in a dictionary

In [102]:
dict_hh = {i:list_hh.count(i) for i in list_hh}


In [103]:
path = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Temp' # use your path
all_files = glob.glob(path + "/*.csv")

list_hhs = []

cnt = 0
finalexcelsheet = pd.DataFrame()
for filename in all_files:
    df = pd.read_csv(filename, sep='comma', index_col=None, header=0, engine='python')
    s1 = filename
    s2 = "T_"
    s= s1[s1.index(s2) + len(s2):]
    split_string = s.split("_")
    household = split_string[0]
    
    list_hhs.append(str(household))

In [104]:
dict_hhs = {i:list_hhs.count(i) for i in list_hhs}


In [105]:
diff_list = [item for item in list_hhs if item not in list_hh]
diff_list

[]

Checking the number of appeariance

In [106]:
diff_dict_hh = dict((k, v) for k, v in dict_hh.items() if v < 2)
key_list = list(diff_dict_hh.keys())
print(key_list)

['001000105AA', '018000238AA', '019000209AA', '019000241AA', '019000620AA', '050000205AA', '051000101AA']


In [107]:
diff_dict_hh = dict((k, v) for k, v in dict_hh.items() if v > 2)
key_list = list(diff_dict_hh.keys())
print(key_list)

['018000293AA']


Checking the difference

In [108]:
list_hh = list(set(list_hh))

print(len(list_hh))

558


In [109]:
print(len(dict_hh))
print(sum(dict_hh.values()))

558
1110


Comparison with serial numbers excel

In [110]:
list_ibutton = pd.read_excel(r'C:\Users\ferha\OneDrive\Desktop\HIGH_Nouna\iButton_serial_numbers (3).xlsx', index_col=0)
list_serial = list_ibutton['Unnamed: 2'].values.tolist()
diff_list = [item for item in list_hh if item not in list_serial]
#diff_list = pd.DataFrame({'col':diff_list})
diff_list = list(set(diff_list))
diff_list

['024000174AA']

In [111]:
list_ibutton = pd.read_excel(r'C:\Users\ferha\OneDrive\Desktop\HIGH_Nouna\iButton_serial_numbers (3).xlsx', index_col=0)
list_serial = list_ibutton['Unnamed: 2'].values.tolist()
diff_list = [item for item in list_hhs if item not in list_serial]
#diff_list = pd.DataFrame({'col':diff_list})
diff_list = list(set(diff_list))
diff_list

['024000174AA']

In [200]:
banco_cont = list_ibutton[((list_ibutton['Unnamed: 3'] == 'Banco' ) | (list_ibutton['Unnamed: 3'] == 'BANCO')) & ((list_ibutton['Unnamed: 4'] == 'CONTROLE') )]

In [201]:
banco_cont


,LISTE DES IBUTTONS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
4,Bagala,044000102AA,Banco,CONTROLE,4000000075CD2541
8,Bagala,044000189AA,Banco,CONTROLE,AB00000075CF1241
13,Bagala,044000311AA,Banco,CONTROLE,680000005EE00741
14,Bagala,044000312AA,Banco,CONTROLE,0300000075CDE141
20,Bagala,044000391AA,Banco,CONTROLE,9500000075C98C41
...,...,...,...,...,...
583,Toni,002000096AA,Banco,CONTROLE,59000000757B8C41
587,Toni,002000170AB,Banco,CONTROLE,9C000000757EA441
591,Toni,002000242AA,Banco,CONTROLE,D300000075785C41
598,Toni,002000389AA,Banco,CONTROLE,6E000000757FE941


In [202]:
banco_cont_id = banco_cont['Unnamed: 2'].values.tolist()

In [203]:
banco_cont_id

['044000102AA',
 '044000189AA',
 '044000311AA',
 '044000312AA',
 '044000391AA',
 '044000419AA',
 '041000102AA',
 '041000222AA',
 '041000293AA',
 '041000298AA',
 '041000336AA',
 '041000338AA',
 '001000001AA',
 '001000096AA',
 '001000112AA',
 '001000122AA',
 '001000126AA',
 '001000154AA',
 '004000018AA',
 '004000140AA',
 '004000221AA',
 '004000234AA',
 '004000238AA',
 '004000261AA',
 '046000016AA',
 '046000039AA',
 '046000109AA',
 '046000121AA',
 '046000154AA',
 '046000157AA',
 '005000062AB',
 '005000073AA',
 '005000151AA',
 '005000198AA',
 '005000199AA',
 '005000202AA',
 '007000386AA',
 '007000400AC',
 '007000570AA',
 '007000622AA',
 '007000772AA',
 '077000423AA',
 '040000144AA',
 '040000156AB',
 '040000157AB',
 '040000237AE',
 '040000339AA',
 '040000453AA',
 '014000023AB',
 '014000024AA',
 '014000045AA',
 '014000046AB',
 '014000062AA',
 '014000157AA',
 '017000119AB',
 '017000458AA',
 '017000504AA',
 '017000613AA',
 '017000678AA',
 '017000688AA',
 '018000075AA',
 '018000100AA',
 '018000

In [204]:
list_col = pd.DataFrame(banco_cont_id)
list_col['var1']=' ,'


#df_col["sql"] = df_col['var'].astype(str) +df_col['cols']+ df_col['var1']

list_col

,0,var1
0,044000102AA,","
1,044000189AA,","
2,044000311AA,","
3,044000312AA,","
4,044000391AA,","
...,...,...
145,002000096AA,","
146,002000170AB,","
147,002000242AA,","
148,002000389AA,","


In [89]:
diff_list1 = [item for item in list_serial if item not in list_hh]
#diff_list = pd.DataFrame({'col':diff_list})
diff_list1

['ID Menage', '024000174AA ', nan, nan, nan]

Merging csv files belong to same householdID

In [124]:
import glob
import os
import pandas

def create_merged_csv(key, filelist):
    outfile = 'Concat_{}.csv'.format(key)
    l = []
    for filename in filelist:
        df = pandas.read_csv(filename)
        l.append(df)
    pd.concat(l).to_csv(outfile, index=False)

def find_filesets(path='.'):
    csv_files = {}
    for name in glob.glob("{}/*_*.csv".format(path)):
        key = '_'.join(name.split('_')[2:-1])
        csv_files.setdefault(key, []).append(name)
        
       
    for key,filelist in csv_files.items(): 
        print(key)
        create_merged_csv(key, filelist)

TEST_DIR_NAME=r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\RH_cleansed'
os.chdir(r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\RH_concat')
find_filesets(TEST_DIR_NAME)

001000001AA
001000004AB
001000005AC
001000011AA
001000030AB
001000089AB
001000092AB
001000096AA
001000098AA
001000105AA
001000111AA
001000112AA
001000114AA
001000120AA
001000122AA
001000126AA
001000138AA
001000140AA
001000148AA
001000154AA
001000155AA
001000176AA
001000183AA
002000012AA
002000021AA
002000021AD
002000027AA
002000028AB
002000058AA
002000096AA
002000100AA
002000102AB
002000169AA
002000170AB
002000170AL
002000194AA
002000235AA
002000242AA
002000244AA
002000311AA
002000320AA
002000330AA
002000362AA
002000373AA
002000389AA
002000408AA
002000428AA
004000053AA
004000056AA
004000060AA
004000065AA
004000080AA
004000084AA
004000108AA
004000140AA
004000144AA
004000165AA
004000177AA
004000183AA
004000190AA
004000191AA
004000210AA
004000221AA
004000234AA
004000238AA
004000248AA
004000254AA
004000260AA
004000261AA
005000019AA
005000054AA
005000056AA
005000062AB
005000072AA
005000073AA
005000109AA
005000126AA
005000139AA
005000145AA
005000149AB
005000151AA
005000160AA
005000168AA
0050

Merging cleansed household data into one table

In [173]:
path = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Pilot\Data_ibutton_05092022\TEMP_CLEANSED\New folder' # use your path
all_files = glob.glob(path + "/*.csv")

for filename in all_files:
    #df = pd.read_csv(filename, sep=',', index_col=None, header=None, engine='python')
    df = pd.read_csv(filename, sep=',', index_col=None, header=0, engine='python')
    df = df.groupby('DateTime').mean().reset_index()
    df = df.drop_duplicates(subset='DateTime', keep='first')
    
    df['DateTime'] = df['DateTime'].astype('datetime64')
    df = df.sort_values(['DateTime'],ascending=False)
    df = df.set_index('DateTime')
    
    
path = r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Pilot\Data_ibutton_05092022\TEMP_CLEANSED' # use your path
all_files = glob.glob(path + "/*.csv")

for filename in all_files:
    df1 = pd.read_csv(filename, sep=',', index_col=None, header=None, engine='python')
    s1 = filename
    s2 = "Concat_"
    s= s1[s1.index(s2) + len(s2):]
    split_string = s.split("_")
    household = split_string[0]
    household = household.rsplit('.', 1)[0]
    if 2 in df1.columns:
    
        df1 = df1.iloc[1:]
        df1.rename(columns = {1: 'first', 2:'second'}, inplace = True)
        
        df1['first'] = df1['first'].replace('nan', np.nan).fillna(0)
        df1['first'] = df1['first'].astype('float64')
        df1['second'] = df1['second'].replace('nan', np.nan).fillna(0)
        df1['second'] = df1['second'].astype('float64')
        #df['first'] = df['first'].apply(lambda x: x.replace(',','.'))
        #df['second'] = df['second'].apply(lambda x: x.replace(',','.'))
        df1[household]= df1['first'] +df1['second']
        
        df1.drop('first', axis=1, inplace=True) 
        df1.drop('second', axis=1, inplace=True) 
        
        df1.rename(columns = {0: 'DateTime'}, inplace = True)
        df1 = df1.groupby('DateTime').mean().reset_index()
        df1 = df1.drop_duplicates(subset='DateTime', keep='first')
        df1['DateTime'] = df1['DateTime'].astype('datetime64')
        df1 = df1.sort_values(['DateTime'],ascending=False)
        df1 = df1.set_index('DateTime')

        df1.info()
        
        
        df =df1.join(df, lsuffix="_left", rsuffix="_right", how='outer')
    else:
        
        df1 = pd.read_csv(filename, sep=',', index_col=None, header=None, engine='python')
        s1 = filename
        s2 = "Concat_"
        s= s1[s1.index(s2) + len(s2):]
        split_string = s.split("_")
        household = split_string[0]
        household = household.rsplit('.', 1)[0]
       
        df1 = df1.iloc[1:]
        
        df1.rename(columns = {1: household}, inplace = True)
        df1[household] = df1[household].replace('nan', np.nan).fillna(0)
        df1[household] = df1[household].astype('float64')
        
        
        df1.rename(columns = {0: 'DateTime'}, inplace = True)
        df1 = df1.groupby('DateTime').mean().reset_index()
        df1 = df1.drop_duplicates(subset='DateTime', keep='first')
        df1['DateTime'] = df1['DateTime'].astype('datetime64')
        df1 = df1.sort_values(['DateTime'],ascending=False)
        df1 = df1.set_index('DateTime')
        
        df1.info()
        df =df1.join(df, how='outer')
        
        
        
df = df.sort_values(['DateTime'],ascending=False)

       
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1731 entries, 2022-09-05 12:30:00 to 2022-08-18 12:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0044AA  1731 non-null   float64
dtypes: float64(1)
memory usage: 27.0 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1731 entries, 2022-09-05 12:45:00 to 2022-08-18 12:15:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0049AA  1731 non-null   float64
dtypes: float64(1)
memory usage: 27.0 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1652 entries, 2022-09-05 13:15:00 to 2022-08-19 08:30:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0064AA  1652 non-null   float64
dtypes: float64(1)
memory usage: 25.8 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1731 entries, 2022-09-05 12:30:00 to 2022-08-18 12:00:00
Data col

,0088AA,0085AA,0084AA,0076AA,0064AA,0049AA,0044AA,0007AA
DateTime,,,,,,,,
2022-09-05 14:15:00,NaN,NaN,33.589,NaN,NaN,NaN,NaN,NaN
2022-09-05 14:00:00,NaN,NaN,34.089,NaN,NaN,NaN,NaN,NaN
2022-09-05 13:45:00,NaN,33.033,33.589,NaN,NaN,NaN,NaN,NaN
2022-09-05 13:30:00,39.619,33.033,33.589,NaN,NaN,NaN,NaN,NaN
2022-09-05 13:15:00,39.619,33.033,33.090,NaN,34.077,NaN,NaN,34.607
...,...,...,...,...,...,...,...,...
2022-08-18 13:00:00,NaN,NaN,34.089,31.081,NaN,35.600,33.139,32.610
2022-08-18 12:45:00,NaN,NaN,NaN,31.081,NaN,36.099,32.640,32.610
2022-08-18 12:30:00,NaN,NaN,NaN,30.582,NaN,35.102,32.141,33.608


In [176]:
df

,DateTime,0088AA,0085AA,0084AA,0076AA,0064AA,0049AA,0044AA,0007AA
0,2022-09-05 14:15:00,NaN,NaN,33.589,NaN,NaN,NaN,NaN,NaN
1,2022-09-05 14:00:00,NaN,NaN,34.089,NaN,NaN,NaN,NaN,NaN
2,2022-09-05 13:45:00,NaN,33.033,33.589,NaN,NaN,NaN,NaN,NaN
3,2022-09-05 13:30:00,39.619,33.033,33.589,NaN,NaN,NaN,NaN,NaN
4,2022-09-05 13:15:00,39.619,33.033,33.090,NaN,34.077,NaN,NaN,34.607
...,...,...,...,...,...,...,...,...,...
1733,2022-08-18 13:00:00,NaN,NaN,34.089,31.081,NaN,35.600,33.139,32.610
1734,2022-08-18 12:45:00,NaN,NaN,NaN,31.081,NaN,36.099,32.640,32.610
1735,2022-08-18 12:30:00,NaN,NaN,NaN,30.582,NaN,35.102,32.141,33.608
1736,2022-08-18 12:15:00,NaN,NaN,NaN,30.582,NaN,35.102,32.640,NaN


In [39]:
cols=df.columns.tolist()
cols.sort(reverse=True)
df=df[cols]

Reseting DateTime as an index

In [175]:
df = df.reset_index('DateTime')

In [67]:
df = df.replace(to_replace=0, value=np.nan)

In case of need for a filter

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7097 entries, 0 to 7096
Columns: 557 entries, DateTime to 001000001AA
dtypes: datetime64[ns](1), float64(556)
memory usage: 30.2 MB


In [209]:
filtered_df = df.loc[(df['DateTime'] >= '2022-01-01 00:00:00')
                     & (df['DateTime'] < '2022-02-01 00:00:00')]
filtered_df

,DateTime,077000423AA,056000216AA,056000210AA,056000157AA,056000151AA,056000132AA,056000111AA,056000108AB,056000103AC,...,001000100AA,001000098AA,001000096AA,001000092AB,001000089AB,001000030AB,001000011AA,001000005AC,001000004AB,001000001AA
8510,2022-01-31 23:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.867,25.260,15.032,27.921,19.425,22.204,21.215,21.343,23.228,21.478
8511,2022-01-31 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,31.231,25.952,15.741,25.881,18.722,21.484,21.215,20.627,21.832,21.478
8512,2022-01-31 23:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.550,24.567,14.321,25.881,18.018,20.762,21.215,21.343,21.132,21.478
8513,2022-01-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.550,24.567,15.032,25.881,17.312,20.038,20.520,19.910,21.832,21.478
8514,2022-01-31 22:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29.867,25.260,15.032,24.513,18.018,19.310,19.124,18.469,19.728,21.478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11481,2022-01-01 01:00:00,NaN,20.107,20.545,27.806,14.114,24.671,23.974,30.343,NaN,...,29.867,32.109,19.267,35.941,27.054,35.395,29.450,31.177,37.515,27.687
11482,2022-01-01 00:45:00,NaN,20.107,19.823,27.806,13.386,23.280,23.276,31.025,NaN,...,29.867,31.431,19.267,35.941,25.680,35.395,29.450,29.099,36.850,28.370
11483,2022-01-01 00:30:00,NaN,20.846,19.823,27.104,13.386,23.280,23.276,31.025,NaN,...,30.550,31.431,18.564,35.941,24.991,35.395,28.772,29.099,36.850,29.051
11484,2022-01-01 00:15:00,NaN,20.846,20.545,27.806,14.840,22.583,23.276,31.705,NaN,...,29.867,32.785,18.564,35.941,24.991,33.373,28.772,29.099,36.850,27.687


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7097 entries, 0 to 7096
Columns: 555 entries, DateTime to 001000001AA
dtypes: datetime64[ns](1), float64(554)
memory usage: 30.1 MB


Saving the csv file

In [177]:
df.to_csv(r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Update Files\Pilot_Temp_JulyAug.csv', index = False)

SQL script

In [130]:

df_col = pd.DataFrame(df.columns, columns=['cols'])
df_col['var1']='` FLOAT NULL,'
df_col['var']='`'
df_col = pd.DataFrame.reindex(df_col,columns = ['var', 'cols','var1'])
df_col['var1'] = df_col['var1'].astype(str)
df_col['var'] = df_col['var'].astype(str)
df_col['cols'] = df_col['cols'].astype(str)
df_col = df_col.iloc[1:]

df_col["sql"] = df_col['var'].astype(str) +df_col['cols']+ df_col['var1']

df_col

,var,cols,var1,sql
1,`,056000216AA,"` FLOAT NULL,","`056000216AA` FLOAT NULL,"
2,`,056000210AA,"` FLOAT NULL,","`056000210AA` FLOAT NULL,"
3,`,056000157AA,"` FLOAT NULL,","`056000157AA` FLOAT NULL,"
4,`,056000151AA,"` FLOAT NULL,","`056000151AA` FLOAT NULL,"
5,`,056000132AA,"` FLOAT NULL,","`056000132AA` FLOAT NULL,"
...,...,...,...,...
554,`,001000030AB,"` FLOAT NULL,","`001000030AB` FLOAT NULL,"
555,`,001000011AA,"` FLOAT NULL,","`001000011AA` FLOAT NULL,"
556,`,001000005AC,"` FLOAT NULL,","`001000005AC` FLOAT NULL,"
557,`,001000004AB,"` FLOAT NULL,","`001000004AB` FLOAT NULL,"


In [131]:
df_col.to_csv(r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Update Files\RH_update_cols.csv', index = False)

,DateTime,077000423AA,056000216AA,056000210AA,056000157AA,056000151AA,056000132AA,056000111AA,056000108AB,056000103AC,...,001000100AA,001000098AA,001000096AA,001000092AB,001000089AB,001000030AB,001000011AA,001000005AC,001000004AB,001000001AA
0,2022-04-30 15:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-04-30 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-04-30 14:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-04-30 14:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-04-30 14:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39076,2022-03-24 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39077,2022-03-24 16:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39078,2022-03-24 16:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39079,2022-03-24 16:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:

df

,DateTime,056000216AA,056000210AA,056000157AA,056000151AA,056000132AA,056000111AA,056000108AB,056000100AA,056000098AB,...,001000105AA,001000098AA,001000096AA,001000092AB,001000089AB,001000030AB,001000011AA,001000005AC,001000004AB,001000001AA
0,2022-08-29 08:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-08-29 08:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-29 08:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-29 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-29 07:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10071,2022-05-16 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,40.515,NaN,NaN,NaN,NaN,NaN,NaN
10072,2022-05-16 10:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,41.162,NaN,NaN,NaN,NaN,NaN,NaN
10073,2022-05-16 10:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,43.094,NaN,NaN,NaN,NaN,NaN,NaN
10074,2022-05-16 10:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,43.094,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
df_col = pd.DataFrame(df.columns)
df_col['var1']=' set'
df_col['var']='update '
df_col['var3']='rh_all '
df_col['var2']='= null where'
df_col['var4']='=0;'
df_col = pd.DataFrame.reindex(df_col,columns = ['var', 'var3','var1', 0,'var2',0,'var4'])
df_col['var1'] = df_col['var1'].astype(str)
df_col['var'] = df_col['var'].astype(str)
df_col[0] = df_col[0].astype(str)
df_col = df_col.iloc[1:]

#df_col["sql"] = df_col['var'].astype(str) +df_col['cols']+ df_col['var1']

df_col

,var,var3,var1,0,var2,0,var4
1,update,rh_all,set,0088AA,= null where,0088AA,=0;
2,update,rh_all,set,0085AA,= null where,0085AA,=0;
3,update,rh_all,set,0084AA,= null where,0084AA,=0;
4,update,rh_all,set,0076AA,= null where,0076AA,=0;
5,update,rh_all,set,0064AA,= null where,0064AA,=0;
6,update,rh_all,set,0049AA,= null where,0049AA,=0;
7,update,rh_all,set,0044AA,= null where,0044AA,=0;
8,update,rh_all,set,0007AA,= null where,0007AA,=0;


In [169]:
df_col.to_csv(r'C:\Users\ferha\OneDrive\Desktop\myPHPadmin\RCT Updates\JulyAugustData\Update Files\null_code_pilot.csv', index = False)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,DateTime,0088AA,0085AA,0084AA,0076AA,0064AA,0060AA,0049AA,0044AA,0025AA,0023AA,0016AA,0007AA
1,3/18/2022 12:30,42.113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3/18/2022 12:15,41.116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3/18/2022 12:00,41.116,37.534,37.584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3/18/2022 11:45,40.617,37.034,37.584,NaN,36.573,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36417,3/3/2021 12:00,NaN,NaN,NaN,37.571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36418,3/3/2021 11:45,NaN,NaN,NaN,37.072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36419,3/3/2021 11:30,NaN,NaN,NaN,37.571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36420,3/3/2021 11:15,NaN,NaN,NaN,37.571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
stats = df.describe()

,007000423AA,056000216AA,056000210AA,056000157AA,056000151AA,056000132AA,056000111AA,056000108AB,056000103AC,056000100AA,...,001000100AA,001000098AA,001000096AA,001000092AB,001000089AB,001000030AB,001000011AA,001000005AC,001000004AB,001000001AA
count,9864.000000,34082.000000,34082.000000,34233.000000,34218.000000,34464.000000,34313.000000,34316.000000,9675.000000,34372.000000,...,32989.000000,32890.000000,32884.000000,32891.000000,33002.000000,33008.000000,32886.000000,32897.000000,33001.000000,32207.000000
mean,29.549316,31.643183,31.929695,30.556302,31.699798,31.763425,31.847567,29.621562,31.021083,30.575938,...,31.290343,32.094823,32.339139,30.027722,31.056408,32.053407,30.290401,31.847138,30.033565,31.364281
std,2.398672,2.826338,3.153405,3.955420,3.029883,4.209863,4.707366,4.217857,3.461252,2.910978,...,4.001775,4.889629,3.064895,4.421735,2.968037,4.429160,3.765040,4.355656,3.883070,3.147722
min,24.619000,23.124444,23.632778,20.684444,24.073889,20.133333,20.635000,19.111667,25.594000,24.086111,...,19.608000,20.573000,23.627000,17.629000,24.064000,20.580000,20.139000,20.635000,19.114000,20.640000
25%,27.619000,29.623000,29.631000,27.687222,29.572778,28.636000,28.636000,27.116667,28.093000,28.085000,...,28.611000,28.574000,30.124000,26.635000,28.563000,28.581000,27.642000,28.636000,27.119000,29.141000
50%,29.117000,31.621000,31.629444,30.186000,31.570556,31.133333,31.135000,29.615000,30.092000,30.084000,...,30.609000,31.572000,32.121000,29.633000,30.562000,31.579000,29.641000,31.633000,29.617000,30.640000
75%,31.115000,33.618333,34.126667,33.181667,34.067222,34.627778,35.131667,32.610556,33.088000,32.581667,...,34.105000,35.569000,34.615000,33.129000,33.059000,35.074000,33.137000,34.628000,32.612000,33.636000
max,36.104000,42.096667,42.111667,42.657222,40.552778,45.590556,46.112222,41.583889,41.072000,41.066667,...,43.085000,46.055000,41.591000,42.604000,40.044000,45.049000,41.117000,45.099000,41.086000,40.120000


If outlier extraction would be needed

In [49]:
#Q1 = df2.quantile(0.05)
#Q3 = df2.quantile(0.95)
#IQR = Q3 - Q1

#df_conf = df2[~((df2 < (Q1 - 1.5 * IQR)) |(df2 > (Q3 + 1.5 * IQR))).any(axis=1)]

low = .05
high = .95
df_conf = df2.quantile([low, high])
df2_filt = df2.apply(lambda x: x[(x>df_conf.loc[low,x.name]) & 
                                    (x < df_conf.loc[high,x.name])], axis=0)

In [50]:
df2_filt.describe()

,0088AA,0085AA,0084AA,0076AA,0064AA,0060AA,0049AA,0044AA,0025AA,0023AA,0016AA,0007AA
count,4132.000000,31104.000000,30593.000000,31393.000000,32712.000000,27790.000000,4147.000000,4167.000000,27582.000000,27469.000000,27455.000000,4109.000000
mean,32.190595,30.373984,30.763082,30.585693,30.410206,30.824959,31.614286,31.321337,31.671774,30.421663,32.181270,30.951308
std,3.846664,3.473219,3.115459,3.424755,3.366798,2.615024,3.917202,3.868121,2.706305,2.740184,2.694697,2.802920
min,25.637000,24.033000,25.594000,24.584000,24.584000,27.113000,25.113000,24.646000,27.136000,26.137000,28.145000,26.113000
25%,29.135000,27.533000,28.093000,28.083000,27.584000,28.612000,28.612000,28.145000,29.135000,28.136000,30.144000,28.612000
50%,31.633000,30.033000,30.592000,30.082000,30.082000,30.611000,31.109000,31.142000,31.608000,30.135000,31.642000,30.611000
75%,35.128000,33.033000,33.090000,33.079000,33.079000,32.610000,34.603000,34.637000,33.605000,32.632000,34.138000,33.109000
max,39.619000,38.083000,37.085000,38.569000,38.070000,36.105000,39.589000,38.627000,37.594500,36.127000,37.629500,36.605000


In [44]:
df2.isnull().sum()

0
0088AA    31727
0085AA     1356
0084AA     1567
0076AA     1356
0064AA       36
0060AA     4907
0049AA    31724
0044AA    31727
0025AA     4946
0023AA     4938
0016AA     4932
0007AA    31719
dtype: int64